## Libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet import preprocess_input


## Preprocess

In [2]:
def load_preprocess(dir, label):
    images = []
    labels = []
    for file in os.listdir(dir):
        img = load_img(os.path.join(dir, file), target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)  
        images.append(img_array)
        labels.append(label)

    return np.array(images), np.array(labels)

sugarcane_leaves_dir = 'sugarcane_leaves'
non_sugarcane_leaves_dir= 'non_sugarcane_leaves'

sugarcane_images, sugarcane_labels = load_preprocess(sugarcane_leaves_dir, 1)
non_sugarcane_images, non_sugarcane_labels = load_preprocess(non_sugarcane_leaves_dir, 0)

X = np.concatenate((sugarcane_images, non_sugarcane_images), axis=0)
y = np.concatenate((sugarcane_labels, non_sugarcane_labels), axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
def mobilenet():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

model = mobilenet()

In [4]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10


78/78 [==============================] - 43s 475ms/step - loss: 0.0206 - accuracy: 0.9866 - val_loss: 3.2680e-05 - val_accuracy: 1.0000
Epoch 2/10
78/78 [==============================] - 26s 335ms/step - loss: 2.0254e-05 - accuracy: 1.0000 - val_loss: 2.0295e-05 - val_accuracy: 1.0000
Epoch 3/10
78/78 [==============================] - 25s 316ms/step - loss: 1.2169e-05 - accuracy: 1.0000 - val_loss: 1.3709e-05 - val_accuracy: 1.0000
Epoch 4/10
78/78 [==============================] - 27s 354ms/step - loss: 8.0317e-06 - accuracy: 1.0000 - val_loss: 1.0138e-05 - val_accuracy: 1.0000
Epoch 5/10
78/78 [==============================] - 25s 318ms/step - loss: 5.6052e-06 - accuracy: 1.0000 - val_loss: 7.6942e-06 - val_accuracy: 1.0000
Epoch 6/10
78/78 [==============================] - 29s 368ms/step - loss: 4.1299e-06 - accuracy: 1.0000 - val_loss: 6.1508e-06 - val_accuracy: 1.0000
Epoch 7/10
78/78 [==============================] - 32s 416ms/step - loss: 3.1394e-06 - accuracy

In [6]:
def image_status(image_path, model):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        return "Sugarcane leaf"
    else:
        return "Not a sugarcane leaf"

new_image_path = 'sugarcane_leaves/healthy_4.jpg'
prediction = image_status(new_image_path, model)


print("Prediction for the image:", prediction)

1/1 [==============================] - 1s 1s/step
Prediction for the image: Sugarcane leaf
